# Quick Start: Running Foundation Model Moirai on BOOM benchmark

This notebook is adapted from the [GiftEval repository](https://github.com/SalesforceAIResearch/gift-eval/tree/main/notebooks) and shows how to run the Moirai model on the BOOM benchmark.

Make sure you download the BOOM benchmark and set the `BOOM` environment variable correctly before running this notebook.

We will use the `Dataset` class from GiftEval to load the data and run the model.

Download BOOM datasets. Calling `download_boom_benchmark` also sets the `BOOM` environment variable with the correct path, which is needed for running the evals below.

In [ ]:
import os
import json
from dotenv import load_dotenv
from dataset_utils import download_boom_benchmark

boom_path = "ChangeMe"
download_boom_benchmark(boom_path)
load_dotenv()

dataset_properties_map = json.load(open("./boom/boom_properties.json"))
all_datasets = list(dataset_properties_map.keys())
print(len(all_datasets))      

In [ ]:
from gluonts.ev.metrics import (
    MAE,
    MAPE,
    MASE,
    MSE,
    MSIS,
    ND,
    NRMSE,
    RMSE,
    SMAPE,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
]

## Moirai Predictor
We load the model with initial hyperparameters, like `prediction_length`, `context_length`, `patch_size`, `num_samples`, etc. You can find more details about these hyperparameters from <https://github.com/SalesforceAIResearch/uni2ts/blob/main/example/moirai_forecast_pandas.ipynb>. 

## Evaluation

Now that we have our predictor class, we can use it to predict on the BOOM datasets. We will use the `evaluate_model` function from `gluonts` to evaluate the model. Then we store the results in a csv file called `all_results.csv` under the `results/{model name}` folder.

The first column in the csv file is the dataset config name which is a combination of the dataset name, frequency and the term:

```python
f"{dataset_name}/{freq}/{term}"
```

In [ ]:
from gluonts.model import evaluate_model
import csv
import os
import time
from gluonts.time_feature import get_seasonality
from gift_eval.data import Dataset
import torch
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

torch.set_float32_matmul_precision("high")

# Iterate over all available datasets
model_name = "moirai_small"
output_dir = f"ChangeMe/{model_name}"
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "all_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MSE[0.5]",
            "eval_metrics/MAE[0.5]",
            "eval_metrics/MASE[0.5]",
            "eval_metrics/MAPE[0.5]",
            "eval_metrics/sMAPE[0.5]",
            "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[0.5]",
            "eval_metrics/mean_weighted_sum_quantile_loss",
            "domain",
            "num_variates",
            "dataset_size",
        ]
    )

for ds_num, ds_name in enumerate(all_datasets):
    print(f"Processing dataset: {ds_name} ({ds_num + 1} of {len(all_datasets)})")
    dataset_term = dataset_properties_map[ds_name]["term"]
    terms = ["short", "medium", "long"]
    for term in terms:
        if (term == "medium" or term == "long") and dataset_term == "short":
            continue
        ds_freq = dataset_properties_map[ds_name]["frequency"]
        ds_config = f"{ds_name}/{ds_freq}/{term}"

        # Initialize the dataset, since Moirai support multivariate time series forecast, it does not require
        to_univariate = False
        dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate,storage_env_var="BOOM")

        # set the Moirai hyperparameter according to each dataset, then create the predictor
        model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-small"),
            prediction_length=1,
            context_length=4000,
            patch_size=32,
            num_samples=100,
            target_dim=1,
            feat_dynamic_real_dim=0,
            past_feat_dynamic_real_dim=0,
        )
        
        model.hparams.prediction_length = dataset.prediction_length
        model.hparams.target_dim = dataset.target_dim
        model.hparams.past_feat_dynamic_real_dim = dataset.past_feat_dynamic_real_dim
        print(dataset.target_dim)
        predictor = model.create_predictor(batch_size=1)

        season_length = get_seasonality(dataset.freq)
        dataset_size = len(dataset.test_data)
        print(f"Dataset size: {dataset_size}")
        
        # forecasts = predictor.predict(dataset.test_data.input)
        res = evaluate_model(
            predictor,
            test_data=dataset.test_data,
            metrics=metrics,
            batch_size=1,
            axis=None,
            mask_invalid_label=True,
            allow_nan_forecast=False,
            seasonality=season_length,
        )

        # Append the results to the CSV file
        with open(csv_file_path, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(
                [
                    ds_config,
                    "moirai_small",
                    res["MSE[mean]"][0],
                    res["MSE[0.5]"][0],
                    res["MAE[0.5]"][0],
                    res["MASE[0.5]"][0],
                    res["MAPE[0.5]"][0],
                    res["sMAPE[0.5]"][0],
                    res["MSIS"][0],
                    res["RMSE[mean]"][0],
                    res["NRMSE[mean]"][0],
                    res["ND[0.5]"][0],
                    res["mean_weighted_sum_quantile_loss"][0],
                    dataset_properties_map[ds_name]["domain"],
                    dataset_properties_map[ds_name]["num_variates"],
                    dataset_size,
                ]
            )

        print(f"Results for {ds_name} have been written to {csv_file_path}")

## Results

Running the above cell will generate a csv file called `all_results.csv` under the `results/moirai_small` folder containing the results for the Moirai model on the boom benchmark. The csv file will look like this:


In [ ]:
import pandas as pd
df = pd.read_csv(output_dir + "/all_results.csv")
df